# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
!pip install beautifulsoup4 requests pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd

def seek_1000_movie_reviews(movie_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    reviews = []
    page = 1
    while len(reviews) < 1000:
        url = f"{movie_url}&start={((page-1)*10)}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Sorry, I Failed to fetch reviews for this page {page}.")
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='text show-more__control')
        if not review_elements:
            break
        reviews.extend([review.get_text(strip=True) for review in review_elements])
        page += 1
    return reviews[:1000]

def put_it_to_csv(reviews, filename):
    df = pd.DataFrame({'Review': reviews})
    df.to_csv(filename, index=False)
    print(f"{len(reviews)} reviews saved to {filename}")

if __name__ == "__main__":
    movie_url = "https://www.imdb.com/title/tt0816692/reviews?ref_=tt_urv"
    reviews = seek_1000_movie_reviews(movie_url)
    if reviews:
        put_it_to_csv(reviews, "interstellar_reviews.csv")
    else:
        print("No reviews was collected.")

1000 reviews saved to interstellar_reviews.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import re

# Downloading the  NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Remove noise
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lowercase all texts
    filtered_tokens = [word.lower() for word in filtered_tokens]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]

    # Join tokens back into a string
    clean_text = ' '.join(lemmatized_tokens)

    return clean_text

# Trying to load the CSV file with the collected reviews
df = pd.read_csv("interstellar_reviews.csv")

# Cleaning the text data and save it in a new column
df['Cleaned_Review'] = df['Review'].apply(clean_text)

# Save the updated DataFrame to a new CSV file
df.to_csv("interstellar_reviews_cleaned.csv", index=False)

print("Data cleaning completed and saved to interstellar_reviews_cleaned.csv")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Data cleaning completed and saved to interstellar_reviews_cleaned.csv


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

nltk.download('averaged_perceptron_tagger')

# Tokenize and POS tag the clean text
tokenized_text = word_tokenize(df['Cleaned_Review'].str.cat(sep=' '))
pos_tags = pos_tag(tokenized_text)

# Calculate the total number of Noun (N), Verb (V), Adjective (Adj), Adverb (Adv)
tag_counts = FreqDist(tag for (word, tag) in pos_tags)
noun_count = tag_counts['NN'] + tag_counts['NNS'] + tag_counts['NNP'] + tag_counts['NNPS']
verb_count = tag_counts['VB'] + tag_counts['VBD'] + tag_counts['VBG'] + tag_counts['VBN'] + tag_counts['VBP'] + tag_counts['VBZ']
adj_count = tag_counts['JJ'] + tag_counts['JJR'] + tag_counts['JJS']
adv_count = tag_counts['RB'] + tag_counts['RBR'] + tag_counts['RBS']

# Print the counts
print(f"Total Nouns (N): {noun_count}")
print(f"Total Verbs (V): {verb_count}")
print(f"Total Adjectives (Adj): {adj_count}")
print(f"Total Adverbs (Adv): {adv_count}")



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Total Nouns (N): 104401
Total Verbs (V): 26280
Total Adjectives (Adj): 39520
Total Adverbs (Adv): 9280


In [ ]:
# Perform dependency parsing and visualize the tree
from spacy import displacy

# Choose one sentence for parsing (you can change the index)
sample_sentence = df['Cleaned_Review'].iloc[0]

# Perform dependency parsing
doc = nlp(sample_sentence)

# Visualize the dependency parsing tree
displacy.render(doc, style='dep', options={'distance': 120, 'compact': True, 'color': 'blue'}, jupyter=True)



In [ ]:
# Perform Named Entity Recognition (NER)
entities = []

for sentence in df['Cleaned_Review']:
    doc = nlp(sentence)
    entities.extend([(ent.text, ent.label_) for ent in doc.ents])

# Count the occurrences of each entity type
entity_counts = FreqDist(entity[1] for entity in entities)

# Print the counts of each entity type
print("Named Entity Recognition (NER) Results:")
for entity_type, count in entity_counts.items():
    print(f"{entity_type}: {count}")



Named Entity Recognition (NER) Results:
PERSON: 3280
ORG: 1720
CARDINAL: 1960
GPE: 400
NORP: 680
DATE: 520
ORDINAL: 1000
TIME: 200
FAC: 80
LOC: 200
PRODUCT: 40
EVENT: 40


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below
#the assignment provided valuable hands-on experience in web scraping and NLP with practical applications